## Required
* forcefield
* initial pdb
* installed gromacs
* installed MDAnalysis

In [2]:
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np

#### Trimming h2a and h2b as it was performed in [JMB](https://linkinghub.elsevier.com/retrieve/pii/S0022283615006956)

In [3]:
nucl=mda.Universe("1kx5.pdb")
dimer=nucl.select_atoms("(segid C and resid 12:119) or (segid D and resid 26:200 )")
dimer.write("h2a_h2b_tr.pdb")

/home/armeev/miniconda3/envs/mdanalysis/lib/python3.6/site-packages/MDAnalysis/topology/guessers.py:72: UserWarning: Failed to guess the mass for the following atom types: MN
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


Performing PDB2GMX

In [4]:
%%bash  --out out --err err
source /usr/local/gromacs/bin/GMXRC
mkdir GMX_system
cd GMX_system
ln -s ../charmm36-jul2017.ff/ .
gmx pdb2gmx -f ../h2a_h2b_tr.pdb -o init.pdb -p topol.top \
-i posre.itp -water tip3p -ff charmm36-jul2017

gmx editconf -bt cubic -d 2 -c -f init.pdb -o init_box.pdb

In [12]:
print('Out:\n',out)
print('Err:\n',err)

Out:
 turning H bonds into constraints...
turning H bonds into constraints...
Analysing residue names:
There are:   205    Protein residues
Analysing Protein...
This run will generate roughly 0 Mb of data

Err:
                       :-) GROMACS - gmx grompp, 2018.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                  

In [8]:
%%bash  --out out --err err
source /usr/local/gromacs/bin/GMXRC
mkdir GMX_run
gmx grompp -f Protocols/minimization.mdp -c GMX_system/init_box.pdb -p GMX_system/topol.top \
-o GMX_run/minimization.tpr -r GMX_system/init_box.pdb
gmx mdrun -deffnm GMX_run/minimization

In [9]:
print('Out:\n',out)
print('Err:\n',err)

Out:
 turning H bonds into constraints...
turning H bonds into constraints...
Analysing residue names:
There are:   205    Protein residues
Analysing Protein...
This run will generate roughly 0 Mb of data

Err:
                       :-) GROMACS - gmx grompp, 2018.3 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                  

In [47]:
%%bash  --out out --err err
source /usr/local/gromacs/bin/GMXRC
gmx grompp -f Protocols/equilibration_vac.mdp -c GMX_run/minimization.gro -p GMX_system/topol.top \
-o GMX_run/equilibration.tpr -r GMX_system/init_box.pdb -v
gmx mdrun -deffnm GMX_run/equilibration

In [ ]:
print('Out:\n',out)
print('Err:\n',err)

In [62]:
%%bash  --out out --err err
source /usr/local/gromacs/bin/GMXRC
gmx grompp -f Protocols/production_vac.mdp -c GMX_run/equilibration.gro -p GMX_system/topol.top \
-o GMX_run/production.tpr -t GMX_run/equilibration.cpt -r GMX_system/init_box.pdb -v
gmx mdrun -deffnm GMX_run/production

In [ ]:
print('Out:\n',out)
print('Err:\n',err)